In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi, cos, sin, sqrt
%matplotlib notebook

fig = plt.figure()

plt.xlabel('x')
plt.ylabel('y')
plt.title('test')
plt.margins(x=0.1,y=0.1,tight=False)

speed = 100.0
ax_max = 500.0/1.5
ay_max = 500.0/1.5
part_max = 0.5
delta_max = 0.2

def gen_test_path():
    x = 5.0
    y = -3.0
    path = []
    last_a = 0
    if 1:
        for l, delta_a, k in [(10, 30, 1), (10, 20,1 ), (10, 150,1.5), (10, -90,1), (10, 20,1), (10, 45,1), (10,0,2), (10, 160,1)]:
            last_a += delta_a
            a1 = last_a / 180.0 * pi
            dx = l * cos(a1)
            dy = l * sin(a1)
            dt = l / speed / k
            path.append([x, y, x + dx, y + dy, dt, 'i'])
            x += dx
            y += dy
    else:
        for i in range(40):
            l, delta_a = 5,9
            last_a += delta_a
            a1 = last_a / 180.0 * pi
            dx = l * cos(a1)
            dy = l * sin(a1)
            dt = l / speed
            path.append([x, y, x + dx, y + dy, dt, 'i'])
            x += dx
            y += dy


    path.append([x, y, x, y, 1, 'l'])

    path[0][5] = 'f'
    
    return path

path = gen_test_path()

#fig = plt.figure()

x = [a for a,b,_,_,_,_ in path]
y = [b for a,b,_,_,_,_ in path]

plt.plot(x,y, "g.-")

In [ ]:
from collections import namedtuple

Quad = namedtuple("Quad", "c_0 c_1 c_2 c_3 c_4")
def quad_x(c, t):
    return (c.c_0*t**4)/24+(c.c_1*t**3)/6+(c.c_2*t**2)/2+c.c_3*t+c.c_4

def quad_v(c, t):
    return (c.c_0*t**3)/6+(c.c_1*t**2)/2+c.c_2*t+c.c_3

def quad_a(c, t):
    return (c.c_0*t**2)/2+c.c_1*t+c.c_2

def quad_j(c, t):
    return c.c_0*t+c.c_1

def quad_jj(c, t):
    return c.c_0


In [ ]:
from math import sqrt, hypot
ax_auto = 1500
ay_auto = 1200
corner_delta = 0.2
acc_delta = 1.0
eps = 0.001

Segment = namedtuple("Segment", "quad_x quad_y dt")

def quad_coeff(v_x0, v_y0, v_x1, v_y1):
    segs = []
    
    if hypot(v_x0, v_y0) < eps or hypot(v_x1, v_y1) < eps:
        print "start/stop, ignoring"
        return segs
    
    v_xm = (-v_x0 + v_x1)/2
    v_ym = (-v_y0 + v_y1)/2
    
    v_m = hypot(v_xm, v_ym)
    x_m = abs(corner_delta * v_xm / v_m)
    y_m = abs(corner_delta * v_ym / v_m)
    
    a_xm = ax_auto
    a_ym = ay_auto
    
    #single step unbound k
    k_list = []
    
    if abs(v_x1 - v_x0) > eps:
        k_x = abs(8*sqrt(ax_auto*x_m)/(3 * (v_x1 - v_x0)))
        print("Using X for K", k_x)
        k_list.append((k_x, v_x0, v_x1, a_xm))

    if abs(v_y1 - v_y0) > eps:
        k_y = abs(8*sqrt(ay_auto*y_m)/(3 * (v_y1 - v_y0)))
        print("Using Y for K", k_y)
        k_list.append((k_y, v_y0, v_y1, a_ym))

    k_list.sort()
    k, b_v0, b_v1, b_am = k_list[0]
    if k > 1.0:
        k = 1.0
        print("Single step")
        
        dt = abs(3 * (b_v1-b_v0) / (2 * b_am))
        
        c_x0 = ( 12 * v_x0 - 12 * v_x1) / dt ** 3
        c_x1 = - (6 * v_x0 - 6 * v_x1) / dt ** 2
        c_x2 = 0
        c_x3 = v_x0
        c_x4 = - (dt * v_x0) / 2

        c_y0 = ( 12 * v_y0 - 12 * v_y1) / dt ** 3
        c_y1 = - (6 * v_y0 - 6 * v_y1) / dt ** 2
        c_y2 = 0
        c_y3 = v_y0
        c_y4 = - (dt * v_y0) / 2
        
        segs.append(Segment(
                Quad(c_x0, c_x1, c_x2, c_x3, c_x4), 
                Quad(c_y0, c_y1, c_y2, c_y3, c_y4),
                dt
            ))
        return segs

    print("Multistep")

    # Main segment
    # multi step unbound k
    k_list = []
    
    if abs(v_x1 - v_x0) > eps:
        k_x = abs(32 / 2 ** (7/2) * sqrt(a_xm) * sqrt(x_m) / (v_x1 - v_x0))
        print("Using X for K", k_x)
        k_list.append((k_x, v_x1 - v_x0, a_xm))

    if abs(v_y1 - v_y0) > eps:
        k_y = abs(32 / 2 ** (7/2) * sqrt(a_ym) * sqrt(y_m) / (v_y1 - v_y0))
        print("Using Y for K", k_x)
        k_list.append((k_y, v_y1 - v_y0, a_ym))

    k_list.sort()
    k, b_v, b_am = k_list[0]
    
    if k > 0.9:
        k = 0.9
        print("Strange, k > 0.9 on multistep")
        
    dt = abs(k*b_v/b_am)
    
    c_x0 = 0
    c_x1 = 0
    c_x2 = - (k * v_x0 - k * v_x1) / dt
    c_x3 = k * v_x0
    c_x4 = - (k * dt * v_x0)/2
    c_x = Quad(c_x0, c_x1, c_x2, c_x3, c_x4)
    
    c_y0 = 0
    c_y1 = 0
    c_y2 = - (k * v_y0 - k * v_y1) / dt
    c_y3 = k * v_y0
    c_y4 = - (k * dt * v_y0)/2
    c_y = Quad(c_y0, c_y1, c_y2, c_y3, c_y4)

    seg_m = Segment(c_x, c_y, dt)  

    # Initial segment
    v_x0 = v_x0
    v_y0 = v_y0
    a_x0 = 0
    a_y0 = 0
    
    v_x1 = quad_v(c_x, 0)
    v_y1 = quad_v(c_y, 0)
    a_x1 = quad_a(c_x, 0)
    a_y1 = quad_a(c_y, 0)
    
    x_1 = quad_x(c_x, 0)
    y_1 = quad_x(c_y, 0)
    
    
    dt_x = abs(6*(v_x1 - v_x0)/a_xm)
    dt_y = abs(6*(v_y1 - v_y0)/a_ym)
    print("dti:", dt_x, dt_y)
    
    
    c_x0 = a_x1 ** 3 / ( 9 * v_x1 ** 2 - 18 * v_x0 * v_x1 + 9 * v_x0 ** 2)
    c_x1 = -a_x1 ** 2 / ( 6 * v_x1 - 6 * v_x0)
    c_x2 = 0
    c_x3 = v_x0
    c_4 = ( a_x1 * x_1 - 6 * v_x0 * v_x1 + 6 * v_x0 ** 2) / a_x1
    
    c_x0 = (-12 * v_x1 + 12 * v_x0 + 6 * a_x1 * dt) / dt ** 3
    c_x1 = - ( -6 * v_x1 + 6 * v_x0 + 2 * a_x1 * dt) / dt ** 2
    c_x2 = 0
    c_x3 = v_x0
    c_x4 = - (-12 * x_1 + 6 * dt * v_x1 + 6 * dt * v_x0 - a_x1 * dt**2) / 12
    c_xi = Quad(c_x0, c_x1, c_x2, c_x3, c_x4)

    c_y0 = (-12 * v_y1 + 12 * v_y0 + 6 * a_y1 * dt) / dt ** 3
    c_y1 = - ( -6 * v_y1 + 6 * v_y0 + 2 * a_y1 * dt) / dt ** 2
    c_y2 = 0
    c_y3 = v_y0
    c_y4 = - (-12 * y_1 + 6 * dt * v_y1 + 6 * dt * v_y0 - a_y1 * dt**2) / 12
    c_yi = Quad(c_y0, c_y1, c_y2, c_y3, c_y4)
    
    dt = max(dt_x, dt_y)

    seg_i = Segment(c_xi, c_yi, dt)  
    
    segs.append(seg_i)
    segs.append(seg_m)
        
    return segs
    
        


In [ ]:
from collections import namedtuple
emu_point = namedtuple("emu_point", "t x y vx vy ax ay")


x1, y1, x2, y2, dt1, _ = path[1]
_, _, x3, y3, dt2, _ = path[2]



lp = path[0][:]
lp[2] = lp[0]
lp[3] = lp[1]

emu_data = []

global_t = 0.0

for cp in path:
    x_0, y_0, x_1, y_1, dt_1, _ = lp
    _, _, x_2, y_2, dt_2, _ = cp
    
    print("lp", lp)
    print("cp", cp)

    lp = cp
    
    v_x0 = (x_1 - x_0)/dt_1
    v_y0 = (y_1 - y_0)/dt_1
    v_x1 = (x_2 - x_1)/dt_2
    v_y1 = (y_2 - y_1)/dt_2

    print('v_x', v_x0, v_x1)
    print('v_y', v_y0, v_y1)

    segs = quad_coeff(v_x0, v_y0, v_x1, v_y1)
    print(segs)
    for seg in segs:
        coefs_x = seg.quad_x
        coefs_y = seg.quad_y
        dt = seg.dt

        coefs_x = coefs_x._replace(c_4=coefs_x.c_4 + x_1)
        coefs_y = coefs_y._replace(c_4=coefs_y.c_4 + y_1)

        subs = 100

        step = dt / subs

        for i in range(subs+1):
            t = i * step

            ddt = 0
            if subs == 100:
                ddt = -0.001

            x = quad_x(coefs_x, t)
            y = quad_x(coefs_y, t)
            vx = quad_v(coefs_x, t)
            vy = quad_v(coefs_y, t)
            ax = quad_a(coefs_x, t)
            ay = quad_a(coefs_y, t)

            emu_data.append(emu_point(t+global_t+ddt, x, y, vx, vy, ax, ay))

        global_t += dt



In [ ]:
fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.x for a in emu_data]
y = [a.y for a in emu_data]

plt.plot(x, y, 'r.-')

xp = [a for a,b,_,_,_,_ in path]
yp = [b for a,b,_,_,_,_ in path]

plt.plot(xp, yp, 'g.-')


fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.vx for a in emu_data]
y = [a.vy for a in emu_data]


plt.plot(t, x, 'r.-')
plt.plot(t, y, "g.-")

fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.ax for a in emu_data]
y = [a.ay for a in emu_data]


plt.plot(t, x, 'r.-')
plt.plot(t, y, "g.-")

In [ ]:
import sympy


In [ ]:
from sympy.solvers import solve
from sympy import Symbol

x = Symbol("x")
y = Symbol("y")
r = Symbol("r")

solve([x**2 + y**2 - r ** 2, x + y], [x, y], dict=True)

